# LangChain Integration

TruLens provides TruChain, a deep integration with LangChain to allow you to inspect and evaluate the internals of your application built using LangChain.

TruChain captures all of the metrics and metadata listed in the [instrumentation overview](../basic_instrumentation). In addition, TruChain instruments the following LangChain classes:

## Instrumented Classes

* langchain.chains.base.Chain
* langchain.vectorstores.base.BaseRetriever
* langchain.schema.BaseRetriever
* langchain.llms.base.BaseLLM
* langchain.prompts.base.BasePromptTemplate
* langchain.schema.BaseMemory
* langchain.schema.BaseChatMessageHistory

## Example Usage

Below is a quick example of usage. First, we'll create a standard LLMChain.

In [ ]:
# required imports
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from trulens_eval import TruChain

# typical langchain setup
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)
chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

To instrument an LLM chain, all that's required is to wrap it using TruChain. Then the instrumented chain can be used like the original.

In [ ]:
# instrument with TruChain
truchain = TruChain(chain)

# call the instrumented application just like the original
llm_response = truchain("What would be a good company name for a company that makes colorful socks?")

You can find the full quickstart available here: [LangChain Quickstart](../langchain_quickstart)

## Async Support

TruChain also provides async support for Langchain through the `acall` method. This allows you to track and evaluate async and streaming LangChain applications.

As an example, below is an LLM chain set up with an async callback.

In [ ]:
import asyncio

from langchain import LLMChain
from langchain import PromptTemplate
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models.openai import ChatOpenAI

from trulens_eval import TruChain
import trulens_eval.utils.python  # makes sure asyncio gets instrumented

# Set up an async callback.
callback = AsyncIteratorCallbackHandler()

# Setup a simple question/answer chain with streaming ChatOpenAI.
prompt = PromptTemplate.from_template("Honestly answer this question: {question}.")
llm = ChatOpenAI(
    temperature=0.0,
    streaming=True, # important
    callbacks=[callback] # callback can be here or below in acall_with_record
)
async_chain = LLMChain(llm=llm, prompt=prompt)

Once you have created the async LLM chain you can instrument it just as before.

In [ ]:
async_tc = TruChain(async_chain)

# Create a task with the call to the chain, but don't wait for it yet.
asyncio.create_task(
    async_tc.acall_with_record(
        inputs=dict(question="What is 1+2? Explain your answer.")
    )
)

# Instead wait for the callback's async generator, getting us each token as it comes in.
async for token in callback.aiter():
    print(token)

For more usage examples, check out the [LangChain examples directory](https://github.com/truera/trulens/tree/main/trulens_eval/examples/frameworks/langchain).